In [158]:
from flask import Flask, render_template, request, redirect, url_for, session, flash
import duckdb
import pandas as pd
import os

In [159]:
app = Flask(__name__)

In [160]:
app.config['SECRET_KEY'] = 'your_very_secret_key'

In [161]:
BASE_DIR = "C:/Users/r0352/Desktop/DBMS_Final_Web"
csv_user_transations_path = os.path.join(BASE_DIR, 'dataset', 'User_transations.csv')
csv_user_card_data_path = os.path.join(BASE_DIR, 'dataset', 'user_card_data.csv')
csv_mcc_with_categories_path = os.path.join(BASE_DIR, 'dataset', 'mcc_with_categories.csv')

csv_user_spending_capability_path = os.path.join(BASE_DIR, 'dataset', 'User_Spending_Capability.csv')
csv_merchant_table_path = os.path.join(BASE_DIR, 'dataset', 'merchant_table.csv')

csv_budget_table_path = os.path.join(BASE_DIR, 'dataset', 'Budget_table.csv')
csv_user_table_path = os.path.join(BASE_DIR, 'dataset', 'user_table.csv')


In [162]:
conn = duckdb.connect()

sql_query = f"SELECT * FROM read_csv_auto('{csv_merchant_table_path}')"
result = conn.sql(sql_query).fetchdf()
print(result)
print(type(result))


     merchant_id     merchant_city merchant_state    zip   mcc
0          59935            Beulah             ND  58523  5499
1          67570        Bettendorf             IA  52722  5311
2          27092             Vista             CA  92084  4829
3          27092       Crown Point             IN  46307  4829
4          13051           Harwood             MD  20776  5813
..           ...               ...            ...    ...   ...
995        81833      Hebbronville             TX  78361  5912
996        22204        Germantown             MD  20874  5541
997        50783           Jarreau             LA  70749  5411
998        71774  Clinton Township             MI  48036  5921
999        32175      Saint Peters             MO  63376  7538

[1000 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>


In [163]:
conn = duckdb.connect()

sql_query = f"""
SELECT m.total_debt, (b.`Spent on Debt` * 12) as debt_paid, m.total_debt - (b.`Spent on Debt` * 12) as debt_remaining
FROM read_csv_auto('{csv_budget_table_path}') AS b
JOIN read_csv_auto('{csv_user_table_path}') AS m ON b.client_id = m.client_id
WHERE m.client_id = 999
"""
result = conn.sql(sql_query).fetchdf()
print(result)

ParserException: Parser Error: syntax error at or near "`"

LINE 2: SELECT m.total_debt, (b.`Spent on Debt` * 12) as debt_paid, m.total_debt - (b.`Spent...
                                ^

In [164]:
@app.route("/")
def home_page():
    return render_template("home-page.html")


In [165]:
@app.route("/user_spend")
def user_spend_page():
    return render_template("user-spend-page.html",title='User Spending Capability')
    

In [166]:
@app.route("/user_spend_input", methods=['POST'])
def user_spend_input():
    user_spend_input = int(request.form.get('client_id'))

    session['user_spend_input'] = user_spend_input
    return render_template("user-spend-page.html",title='User Spending Capability')
    

In [167]:
@app.route("/user_spend_output")
def user_spend_output():
    user_spend_input = session.pop('user_spend_input', [])
    conn = duckdb.connect()
    
    sql_query = f"SELECT * FROM read_csv_auto('{csv_user_spending_capability_path}') WHERE Client_ID={user_spend_input}"
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='User Spending Capability')
    

In [168]:
@app.route("/merchant_analysis")
def merchant_analysis_page():
    return render_template("merchant-analysis-page.html",title='Merchant Table')
    

In [169]:
@app.route("/merchant_analysis_input", methods=['POST'])
def merchant_analysis_input():
    merchant_analysis_input = int(request.form.get('merchant_id'))

    session['merchant_analysis_input'] = merchant_analysis_input
    return render_template("merchant-analysis-page.html",title='Merchant Table')
    

In [170]:
@app.route("/merchant_analysis_output")
def merchant_analysis_output():
    merchant_analysis_input = session.pop('merchant_analysis_input', [])
    conn = duckdb.connect()
    
    sql_query = f"SELECT * FROM read_csv_auto('{csv_merchant_table_path}') WHERE merchant_id={merchant_analysis_input}"
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='Merchant Table')
    

In [171]:
@app.route("/user_update", methods=["GET", "POST"])
def user_update():
    if request.method == "POST":
        action_type = request.form.get("action_type")

        if action_type == "new_transaction":
            # Read values from the HTML form
            card_id = request.form.get("card_id")
            amount = request.form.get("amount")
            merchant_id = request.form.get("merchant_id")
            mcc = request.form.get("mcc")

            try:
                cur = mysql.connection.cursor()

                # 🔴 YOUR SQL, but with parameters:
                cur.execute("""
                    INSERT INTO user_transations (card_id, date, amount, merchant_id, mcc)
                    VALUES (%s, DATE_FORMAT(NOW(), '%%Y-%%m-%%d %%H:%%i:%%s'), %s, %s, %s);
                """, (card_id, amount, merchant_id, mcc))

                mysql.connection.commit()
                cur.close()

                flash("Transaction successfully inserted!", "success")
            except Exception as e:
                mysql.connection.rollback()
                flash(f"Error inserting transaction: {e}", "danger")

            # After POST, redirect back so refresh doesn’t resubmit
            return redirect(url_for("user_update"))

    # GET → just show the form
    return render_template("user_update.html")

In [172]:
@app.route("/user_card")
def user_card():
    return render_template('user-card-page.html', title='User Card Data')

In [173]:
@app.route("/user_card_input", methods=['POST'])
def user_card_input():
    user_card_input = int(request.form.get('card_id'))

    session['user_card_input'] = user_card_input
    return render_template('user-card-page.html', title='User Card Data')

In [174]:
@app.route("/user_card_output")
def user_card_output():
    user_card_input = session.pop('user_card_input', [])
    conn = duckdb.connect()
    
    sql_query = f"SELECT * FROM read_csv_auto('{csv_user_card_data_path}') WHERE card_id={user_card_input}"
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='User Card Data')

In [175]:
@app.route("/user_categorized_spend")
def user_categorized_spend():
    return render_template('user-categorized-spend-page.html', title='Query: User Categorized Spend')

In [176]:
@app.route("/user_categorized_spend_input", methods=['POST'])
def user_categorized_spend_input():
    user_categorized_spend_input = int(request.form.get('client_id'))

    session['user_categorized_spend_input'] = user_categorized_spend_input
    return render_template('user-categorized-spend-page.html', title='Query: User Categorized Spend')

In [177]:
@app.route("/user_categorized_spend_output")
def user_categorized_spend_output():
    user_categorized_spend_input = session.pop('user_categorized_spend_input', [])
    conn = duckdb.connect()
    
    sql_query = f"""
    SELECT m.Category, SUM(t.amount) AS total_spend
    FROM read_csv_auto('{csv_user_transations_path}') AS t
    JOIN read_csv_auto('{csv_user_card_data_path}') AS c ON t.card_id = c.card_id
    JOIN read_csv_auto('{csv_mcc_with_categories_path}') AS m ON t.mcc = m.MCC_code
    WHERE c.client_id = {user_categorized_spend_input}
    GROUP BY m.Category
    ORDER BY total_spend DESC;
    """ 
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='Query: User Categorized Spend')

In [178]:
@app.route("/max_spend_category")
def max_spend_category_page():
    return render_template("max-spend-category-page.html",title='Maximum Spending Category')
    

In [179]:
@app.route("/max_spend_category_output")
def max_spend_category_output():
    conn = duckdb.connect()
    
    sql_query = f"""
    SELECT m.Category, SUM(t.amount) AS total_spend_perCategory
    FROM read_csv_auto('{csv_user_transations_path}') AS t
    JOIN read_csv_auto('{csv_mcc_with_categories_path}') AS m ON t.mcc = m.MCC_code
    GROUP BY m.Category
    ORDER BY total_spend_perCategory DESC;
    """ 
    result_data = conn.sql(sql_query).fetchdf()
    
    html_web_table = result_data.to_html(classes='table table-stripped')
    return render_template('query-result-page.html', table_content=html_web_table, title='Maximum Spending Category')

In [180]:
if __name__ == '__main__':
    app.run( )

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Dec/2025 21:52:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:52:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Dec/2025 21:52:56] "GET /user_spend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:52:59] "POST /user_spend_input HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:00] "GET /user_spend_output HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:02] "GET /user_spend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:05] "POST /user_spend_input HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:06] "GET /user_spend_output HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:08] "GET /user_categorized_spend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:14] "POST /user_categorized_spend_input HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:16] "GET /user_categorized_spend_output HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/2025 21:53:25] "GET /user_categorized_spend HTTP/1.1" 200 -
127.0.0.1 - - [01/Dec/